In [1]:
import os
import re
from tqdm import tqdm

from bdv_toolz.cli import nd2_to_bdv
import sys

from bdv_toolz.bdv_creation import create_bdv_n5_multi_tile


/home/thomas/miniconda3/envs/preprocess-img/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
xml_path = "/mnt/gdrive/ThNe/development_retina/raw_images_n5"
from io_images import get_images_infoframe
df = get_images_infoframe(xml_path, extension= ".xml", conditions=['Age', 'Sex', 'Side'])

folder_path <class 'str'> /mnt/gdrive/ThNe/development_retina/raw_images_n5
Found 9 files...
There are 9 files in folder_location /mnt/gdrive/ThNe/development_retina/raw_images_n5


In [14]:
df

,Age,Sex,Side,file_name,file_path
0,P10,M,L,retina_Age_P10_Sex_M_Side_L_Animal_1.xml,/mnt/gdrive/ThNe/development_retina/raw_images...
1,P10,M,R,retina_Age_P10_Sex_M_Side_R_Animal_1.xml,/mnt/gdrive/ThNe/development_retina/raw_images...
2,P15,M,L,retina_Age_P15_Sex_M_Side_L_Animal_2.xml,/mnt/gdrive/ThNe/development_retina/raw_images...
3,P15,M,L,retina_Age_P15_Sex_M_Side_L_Animal_1.xml,/mnt/gdrive/ThNe/development_retina/raw_images...
4,P15,M,R,retina_Age_P15_Sex_M_Side_R_Animal_2.xml,/mnt/gdrive/ThNe/development_retina/raw_images...
5,P15,M,R,retina_Age_P15_Sex_M_Side_R_Animal_1.xml,/mnt/gdrive/ThNe/development_retina/raw_images...
6,P15,F,R,retina_Age_P15_Sex_F_Side_R_Animal_2.xml,/mnt/gdrive/ThNe/development_retina/raw_images...
7,P21,F,R,retina_Age_P21_Sex_F_Side_R_Animal_1.xml,/mnt/gdrive/ThNe/development_retina/raw_images...
8,P21,F,L,retina_Age_P21_Sex_F_Side_L_Animal_1.xml,/mnt/gdrive/ThNe/development_retina/raw_images...


In [ ]:
import xxhash
import os
import time

def files_match(src_path, dest_path):
    """Compare two files by size first, then by xxhash if needed."""
    
    if os.path.getsize(src_path) != os.path.getsize(dest_path):
        return False  # Files are definitely different
    
    # Use xxhash for comparison
    return compute_xxhash(src_path) == compute_xxhash(dest_path)

def compute_xxhash(file_path, chunk_size=65536):
    """Compute xxhash checksum for a given file with progress tracking."""
    hasher = xxhash.xxh64()
    
    # Get file size and calculate total number of chunks
    file_size = os.path.getsize(file_path)
    total_chunks = (file_size + chunk_size - 1) // chunk_size  # This is equivalent to math.ceil(file_size / chunk_size)
    
    print(f"Total number of chunks to process: {total_chunks}")
    
    total_processed_chunks = 0
    
    try:
        with open(file_path, "rb") as f:
            while chunk := f.read(chunk_size):
                hasher.update(chunk)
                total_processed_chunks += 1
                
                # Print progress every 1000 chunks
                if total_processed_chunks % 1000 == 0:
                    print(f"Chunks processed: {total_processed_chunks}/{total_chunks}")
        
        return hasher.hexdigest()
    
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None


In [3]:
import multiprocessing

num_cpus = multiprocessing.cpu_count()
print(f"Number of CPUs: {num_cpus}")


Number of CPUs: 8


In [2]:
import psutil

cpu_percent = psutil.cpu_percent(percpu=True)  # Get usage per CPU core
print(f"CPU usage per core: {cpu_percent}")


CPU usage per core: [23.3, 15.7, 15.8, 22.3, 22.2, 19.3, 15.4, 29.8]


In [ ]:
    total_cpus = multiprocessing.cpu_count()


In [1]:
import psutil

num_active_cpus = len([p for p in psutil.Process().cpu_affinity()])
print(f"Number of CPUs used by this process: {num_active_cpus}")


Number of CPUs used by this process: 8


In [ ]:
import psutil
import os

pid = os.getpid()  # Get current Python process ID
process = psutil.Process(pid)

# Get number of CPU cores used by this process
num_used_cpus = len(process.cpu_affinity())

print(f"Number of CPUs used by this Python process: {num_used_cpus}")


In [ ]:

# Define base paths
base_src_folder_path = "/run/user/1001/gvfs/smb-share:server=scratch4.ista.ac.at,share=scratch/siegegrp/_ImageDrop/NIKON-W1-03/Thomas/nd_seq"
base_dest_folder_path = "/mnt/gdrive/ThNe/_Bioimaging/development_retina"

# Define file names
src_file = "slide_P10_M_1001.nd2"
dest_file = "slide_P10_M_1001.nd2"

# Construct full file paths
src_path = os.path.join(base_src_folder_path, src_file)
dest_path = os.path.join(base_dest_folder_path, dest_file)

# Check if files match
if files_match(src_path, dest_path):
    print("✅ Files match!")
else:
    print("❌ Files are different!")
